<a href="https://colab.research.google.com/github/Nickguild1993/PyLearn_DS/blob/main/GroupBy_with_Agg()_Review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Going over Groupby and the subsequent operations used with it

In [5]:
# import libraries

import pandas as pd
import numpy as np
import seaborn as sns # tips dataset

Grab the dataset

In [6]:
tips = sns.load_dataset("tips")
tips.head(5)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


#### Group By

The groupby itself isn't that important, it's what you do with it after it's created.

Can do three types of operations with it, most important being agg (aggregate)

In [7]:
tips_gb = tips.groupby(["sex", "smoker"])
tips_gb

####  Agg (Aggregate)

Aggregrates the data in these groups into one value. Use a dictionary to specify which values you'd like.

In [8]:
tips_agg = tips_gb.agg({
    "tip" : ["mean", "min"], # get mean and min of the "tip" column
    "day" : "first", # first gets the first value by index ordering
    "total_bill" : "size" # "size" gets you the total number of bills in the group
})

tips_agg

tip         day total_bill
                   mean   min first       size
sex    smoker                                 
Male   Yes     3.051167  1.00   Sat         60
       No      3.113402  1.25   Sun         97
Female Yes     2.931515  1.00   Sat         33
       No      2.773519  1.00   Sun         54

In [11]:
tippin = tips_gb.agg({
    "tip" : ["min", "max", "mean"],
    "day" : "first",
    "total_bill" : "size",
})

tippin

tip                   day total_bill
                min   max      mean first       size
sex    smoker                                       
Male   Yes     1.00  10.0  3.051167   Sat         60
       No      1.25   9.0  3.113402   Sun         97
Female Yes     1.00   6.5  2.931515   Sat         33
       No      1.00   5.2  2.773519   Sun         54

output has weird structure -> multi index.

#### use reset_index() to get rid of multi-indexing

In [12]:
tippin.reset_index()

sex smoker   tip                   day total_bill
                   min   max      mean first       size
0    Male    Yes  1.00  10.0  3.051167   Sat         60
1    Male     No  1.25   9.0  3.113402   Sun         97
2  Female    Yes  1.00   6.5  2.931515   Sat         33
3  Female     No  1.00   5.2  2.773519   Sun         54

In [13]:
tippin.columns

MultiIndex([(       'tip',   'min'),
            (       'tip',   'max'),
            (       'tip',  'mean'),
            (       'day', 'first'),
            ('total_bill',  'size')],
           )

stacking is confusing but useable

In [14]:
tippin.stack()

day        tip  total_bill
sex    smoker                                  
Male   Yes    first  Sat        NaN         NaN
              max    NaN  10.000000         NaN
              mean   NaN   3.051167         NaN
              min    NaN   1.000000         NaN
              size   NaN        NaN        60.0
       No     first  Sun        NaN         NaN
              max    NaN   9.000000         NaN
              mean   NaN   3.113402         NaN
              min    NaN   1.250000         NaN
              size   NaN        NaN        97.0
Female Yes    first  Sat        NaN         NaN
              max    NaN   6.500000         NaN
              mean   NaN   2.931515         NaN
              min    NaN   1.000000         NaN
              size   NaN        NaN        33.0
       No     first  Sun        NaN         NaN
              max    NaN   5.200000         NaN
              mean   NaN   2.773519         NaN
              min    NaN   1.000000         NaN
              size   NaN        NaN        54.0

INSTEAD USE THIS

In [15]:
tippin.columns = ["__".join(col).strip() for col in tippin.columns.values]
tippin.columns

Index(['tip__min', 'tip__max', 'tip__mean', 'day__first', 'total_bill__size'], dtype='object')

#### FILTERING

Filtering out members of groups that don't meet our criteria

In this example, trying to find the least busy times the resturuant is open.  
- do this by exlcuding all times above the median 

In [16]:
# using same groupby syntax as aggregrate 

tips_gb = tips.groupby(["day", "time"])

In [19]:
median_size = tips_gb.agg({"size": "sum"}).median()[0]
median_size

20.0

In [20]:
tips_gb.filter(lambda group: group['size'].sum() < median_size).head()

,total_bill,tip,sex,smoker,day,time,size
220,12.16,2.20,Male,Yes,Fri,Lunch,2
221,13.42,3.48,Female,Yes,Fri,Lunch,2
222,8.58,1.92,Male,Yes,Fri,Lunch,1
223,15.98,3.00,Female,No,Fri,Lunch,3
224,13.42,1.58,Male,Yes,Fri,Lunch,2
